In [ ]:
import urllib.request
import zipfile
import io

url = "https://github.com/Mulherbe/ia_image/raw/main/train.zip"
with urllib.request.urlopen(url) as url_response:
    with zipfile.ZipFile(io.BytesIO(url_response.read())) as zip_file:
        zip_file.extractall()
        print("Images extraites avec succès.")

urlTest = "https://github.com/Mulherbe/ia_image/raw/main/test.zip"
with urllib.request.urlopen(urlTest) as url_response:
    with zipfile.ZipFile(io.BytesIO(url_response.read())) as zip_file:
        zip_file.extractall()
        print("Images extraites avec succès.")

urlVal= "https://github.com/Mulherbe/ia_image/raw/main/val.zip"
with urllib.request.urlopen(urlVal) as url_response:
    with zipfile.ZipFile(io.BytesIO(url_response.read())) as zip_file:
        zip_file.extractall()
        print("Images extraites avec succès.")

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/train'
validation_dir = '/content/val'

# Dimensions des images d'entrée
img_height = 224
img_width = 224

# Création de l'objet ImageDataGenerator pour augmenter les données d'entraînement
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Création de l'objet ImageDataGenerator pour normaliser les données de validation
validation_datagen = ImageDataGenerator(rescale=1./255)

# Création des générateurs d'images à partir des répertoires d'entraînement et de validation
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=32,
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              target_size=(img_height, img_width),
                                                              batch_size=32,
                                                              class_mode='binary')

# Création du modèle
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilation du modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entraînement du modèle
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=validation_generator)


In [ ]:
from google.colab import drive
model.save('/content/drive/MyDrive/ia_modelV2.h5')

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# Charger le modèle entraîné
model = load_model('/content/drive/MyDrive/ia_modelV1.h5')

# Définir les noms de classes
class_names = ['normal', 'pneumonia']

# Préparer les données de test
test_datagen = ImageDataGenerator(rescale=1./255)
test_dir = '/content/test'
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=1,
        class_mode=None,
        shuffle=False)

# Effectuer des prédictions sur chaque image de test
predictions = model.predict(test_generator)

# Récupérer les noms de fichier de test et les labels correspondants
test_filenames = test_generator.filenames
test_labels = test_generator.classes


# Afficher chaque image de test avec sa prédiction
fig = plt.figure(figsize=(20, 20))
for i in range(len(predictions)):
    filename = test_filenames[i]
    prediction = predictions[i][0]
    label = test_labels[i]
    if abs(prediction - label) < 0.5:
        color = 'green' # prédiction correcte en vert
    else:
        color = 'red' # prédiction incorrecte en rouge
    img = plt.imread(os.path.join(test_dir, filename))
    fig.add_subplot(108, 6, i+1)
    plt.title(f"{class_names[label]} : {class_names[int(prediction + 0.5)]} ({prediction:.2f})", color=color)
    plt.axis('off')
plt.show()
# Évaluer les performances du modèle sur l'ensemble de test
loss, accuracy = model.evaluate(test_generator)

# Afficher la précision du modèle sur l'ensemble de test
print('Accuracy on test set:', accuracy)